# $T_\phi$ characterization

T is defined as by T_2* when removing the effect of T1, or more precisely:
$1/T_2* = 1/2T_1 + 1/T_\phi.$
We therefore create a composite experiment consisting of a $T_1$ experiment and a $T_2* $ experiment. From the results of these two, we compute the results for T$_\phi.$

In [1]:
import numpy as np
import qiskit
from qiskit_experiments.library.characterization import Tphi
from qiskit_experiments.library.characterization import TphiAnalysis

In [2]:
# An Aer simulator
from qiskit.test.mock import FakeVigo
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel

# Create a pure relaxation noise model for AerSimulator
noise_model = NoiseModel.from_backend(
    FakeVigo(), thermal_relaxation=True, gate_error=False, readout_error=False
)

# Create a fake backend simulator
backend = AerSimulator.from_backend(FakeVigo(), noise_model=noise_model)

# Look up target T1 of qubit-0 from device properties
qubit0_t1 = backend.properties().t1(0)
print("T1 for FakeVigo = " + str(qubit0_t1))

# Time intervals to wait before measurement
delays_t1 = np.arange(1e-6, 3 * qubit0_t1, 1e-5)
print(delays_t1)

# set the desired delays
delays_t2 = list(np.arange(1e-6, 50e-6, 2e-6))
print(delays_t2)
qubit = 0



T1 for FakeVigo = 0.00012170801410836628
[1.00e-06 1.10e-05 2.10e-05 3.10e-05 4.10e-05 5.10e-05 6.10e-05 7.10e-05
 8.10e-05 9.10e-05 1.01e-04 1.11e-04 1.21e-04 1.31e-04 1.41e-04 1.51e-04
 1.61e-04 1.71e-04 1.81e-04 1.91e-04 2.01e-04 2.11e-04 2.21e-04 2.31e-04
 2.41e-04 2.51e-04 2.61e-04 2.71e-04 2.81e-04 2.91e-04 3.01e-04 3.11e-04
 3.21e-04 3.31e-04 3.41e-04 3.51e-04 3.61e-04]
[1e-06, 3e-06, 5e-06, 7.000000000000001e-06, 9.000000000000002e-06, 1.1000000000000003e-05, 1.3000000000000003e-05, 1.5000000000000002e-05, 1.7000000000000003e-05, 1.9000000000000004e-05, 2.1000000000000006e-05, 2.3000000000000003e-05, 2.5000000000000005e-05, 2.7000000000000006e-05, 2.9000000000000004e-05, 3.1e-05, 3.3e-05, 3.5000000000000004e-05, 3.7000000000000005e-05, 3.9000000000000006e-05, 4.100000000000001e-05, 4.3e-05, 4.5e-05, 4.7000000000000004e-05, 4.9000000000000005e-05]


In [3]:
# Create an experiment for qubit 0 with the specified time intervals
exp = Tphi(qubit=0, delays_t1=delays_t1, delays_t2=delays_t2, osc_freq=1e5)
# Set scheduling method so circuit is scheduled for delay noise simulation
exp.set_transpile_options(scheduling_method='asap')

expdata = exp.run(backend=backend, analysis=TphiAnalysis()).block_for_results()
result = expdata.analysis_results("T_phi")

in t2ramsey _set_backend


In [4]:
# Print the result for T_phi
print(result)

DbAnalysisResultV1
- name: T_phi
- value: 1.1713979863091273e-05 ± 3.1442018893973085e-05
- quality: bad
- device_components: ['Q0']
- verified: False


In [5]:
# It is possible to see the results of the sub-experiments:
print(expdata)

---------------------------------------------------
Experiment: Tphi
Experiment ID: 8c52fb0e-3ab3-47a9-8550-1a2d65e1438d
Child Experiment Data: 2
Status: ExperimentStatus.DONE
Backend: aer_simulator(fake_vigo)
Data: 62
Analysis Results: 1
Figures: 0


In [6]:
print(expdata.child_data(0).analysis_results("T1"))

DbAnalysisResultV1
- name: T1
- value: 0.00012318856580158585 ± 2.2075141225835903e-06 s
- χ²: 1.4066670765740719
- quality: good
- device_components: ['Q0']
- verified: False


In [7]:
print(expdata.child_data(1).analysis_results("T2star"))

DbAnalysisResultV1
- name: T2star
- value: 1.1182317523178244e-05 ± 2.865266987893251e-05 s
- χ²: 1.0430271249496532
- quality: bad
- device_components: ['Q0']
- verified: False
